In [ ]:
from face_alignment import FaceAlignment, LandmarksType

from loreal_poc.dataloaders.loaders import DataLoader300W
from loreal_poc.dataloaders.wrappers import CroppedDataLoader, CachedDataLoader
from loreal_poc.marks.facial_parts import FacialParts
from loreal_poc.visualisation.draw import draw_marks
from loreal_poc.tests.performance import NMEMean
from loreal_poc.models.wrappers import FaceAlignmentWrapper
from loreal_poc.tests.base import Test
import numpy as np

Benchmark

In [ ]:
ds = DataLoader300W(dir_path="300W/sample")

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

L'Oreal

In [ ]:
chosen_idx = 4
facial_part = FacialParts.upper_half
cropped_ds = CroppedDataLoader(ds, facial_part, crop_img=False)
image, picked_landmarks, meta = cropped_ds[chosen_idx]

In [ ]:
draw_marks(image, [picked_landmarks], ["green"], [True])

### Model 

In [ ]:
model = FaceAlignmentWrapper(model=FaceAlignment(LandmarksType.TWO_D, device=device, flip_input=False))

### NME test on original dataset

In [ ]:
test_on_original_ds = Test(metric=NMEMean, threshold=1).run(model, ds)
test_on_original_ds

## Model predictions on cropped image

In [ ]:
cropped_ds = CroppedDataLoader(ds, facial_part, crop_img=False)
cropped_image, _, _ = cropped_ds[chosen_idx]
predictions = model.predict(cropped_ds, [chosen_idx], facial_part=facial_part)
draw_marks(cropped_image, [predictions.prediction[0]], ["red"], [True])

## NME test on all cropped images



### rel_margin range [0.01, 0.1]

In [ ]:
rel_margins = np.arange(0.0, 0.1, step=0.025)

metrics, predictions = [], []
for rel_margin in rel_margins:
    _test_result = Test(metric=NMEMean, threshold=1).run(
        model, CroppedDataLoader(ds, part=facial_part, margins=rel_margin)
    )
    metrics.append(_test_result.metric_value)
    predictions.append(_test_result.prediction_results[0].prediction)

In [ ]:
from loreal_poc.transformation_functions.crop import get_boundaries_from_marks, crop_mark

colors = ["red", "pink", "magenta", "blue"]
squares = []
for rel_margin in rel_margins:
    img, marks, meta = ds[chosen_idx]
    h, w, _ = img.shape
    margins = np.array([w, h]) * rel_margin
    cropped_mark = crop_mark(marks, facial_part)
    left, upper, right, lower = get_boundaries_from_marks(cropped_mark, margins)

    squares.append([(left, lower), (right, upper)])

draw_marks(image, [p[chosen_idx] for p in predictions], colors, squares=squares)

In [ ]:
draw_marks(image, [p[chosen_idx] for p in predictions], colors, squares=squares)

In [ ]:
draw_marks(image, np.array(predictions)[:, chosen_idx], colors)

In [ ]:
np.array(predictions)[:, chosen_idx].shape

In [ ]:
import matplotlib.pyplot as plt

# Plotting the Graph
plt.plot(rel_margins, metrics, label=f"cropped dataset ({facial_part.name})")
for i, p in enumerate(predictions):
    plt.plot(rel_margins[i], metrics[i], "o", c=colors[i])
plt.plot(rel_margins, [test_on_original_ds.metric_value] * len(rel_margins), "g--", label="original dataset")
plt.title(f"NME on {facial_part.name} w.r.t. rel_margin")
plt.xlabel("rel_margin")
plt.ylabel(f"NME on {facial_part.name}")
plt.legend()
plt.show()

In [ ]:
from loreal_poc.tests.base import TestDiff
from loreal_poc.tests.performance import NMEMean, NMEStd

TestDiff(NMEMean, threshold=1).run(model, ds, CroppedDataLoader(ds, part=facial_part, margins=0.01))

In [ ]:
TestDiff(NMEStd, threshold=1).run(model, ds, CroppedDataLoader(ds, part=facial_part, margins=0.01))